In [1]:
import requests
from bs4 import BeautifulSoup
import html5lib
import re
from urllib.parse import urljoin
import sqlite3
import pandas as pd
import numpy as np

In [2]:
# base url
# there are a bunch of "makes" (manufacturers) which are buttons
# all associated with links to the catalogue
# we need to get the name of the make and the link
# let's build a base function which creates the soup of html
base = 'https://www.urparts.com/'
extension = 'index.cfm/page/catalogue'

def get_soup(cur_url):
    s      = requests.session()
    r      = s.get(cur_url)
    soup   = BeautifulSoup(r.text,"html5lib")
    return(soup)

soup = get_soup(urljoin(base,extension))

In [3]:
# now we bild a function that will get all the links
# filters them by a specific link string
def get_links(soup,link_string,final_layer=False):
    """
    INPUT:
    soup:        soup oject of bs4 of html
    link_string: tuple of words
    final_layer: if True then the critical string gets '?part=' attached
                 otherwise: '/'; default: False
    
    OUTPUT:
    links_dict: dictionary that has as keys the names of the links and as values
                the actual url's; the keys are tuples representing the layers of
                the website; link_string becomes longer by adding the name of the link
                but the first elemnt of the tuple is cut off (here: "catalogue")
                because it is the seed website and is the same for all tuples; and thus
                of no informational value
    """
    # make the critical string to filter links
    if final_layer:
        critical_string = link_string[0] + '?part='
    else:
        critical_string = '/'.join(link_string)
        critical_string = critical_string + '/'
    
    links = soup.find_all("li")
    links_dict = {}
    for l in links:
        if l.find('a') is not None:
            cur_text = l.get_text().strip()
            cur_url  = l.a['href']
            
            if cur_url.find(critical_string) is not -1:
                # make the key for the cur_url
                str_list = list(link_string)
                str_list.append(cur_text)
                new_key = tuple(str_list[1:])
                links_dict[new_key] = cur_url
    return(links_dict)

manufacturers = get_links(soup,link_string=tuple(['catalogue']))
print(manufacturers)

{('Ammann',): 'index.cfm/page/catalogue/Ammann', ('Atlas',): 'index.cfm/page/catalogue/Atlas', ('Atlas-Copco',): 'index.cfm/page/catalogue/Atlas-Copco', ('Bell',): 'index.cfm/page/catalogue/Bell', ('Bomag',): 'index.cfm/page/catalogue/Bomag', ('Doosan',): 'index.cfm/page/catalogue/Doosan', ('FAI',): 'index.cfm/page/catalogue/FAI', ('Hitachi',): 'index.cfm/page/catalogue/Hitachi', ('Hyundai',): 'index.cfm/page/catalogue/Hyundai', ('Isuzu',): 'index.cfm/page/catalogue/Isuzu', ('JCB',): 'index.cfm/page/catalogue/JCB', ('Kawasaki',): 'index.cfm/page/catalogue/Kawasaki', ('Komatsu',): 'index.cfm/page/catalogue/Komatsu', ('Mitsubishi',): 'index.cfm/page/catalogue/Mitsubishi', ('Moxy',): 'index.cfm/page/catalogue/Moxy', ('Volvo',): 'index.cfm/page/catalogue/Volvo'}


In [4]:
# now we define a function that can be applied recursively
# building a dictionary and then going a layer deeper and thereby
# updating the dictionary
# it thus digs into the links that fullfill a certain criterion
# the criterion tuple will be growing starting with 'catalogue'
# and the subsequent names of the links

# the function can take as input a base-link (start of recursion)
# or a dictionary to dig into
base = 'https://www.urparts.com/'
extension = 'index.cfm/page/catalogue'

def scrape(base=None, scrape_dict=None,extension=extension,condition_seed='catalogue',final_layer=False):
    """
    INPUT:
    base:        base website to scrape
    extension:   extension from base that yields the website to scrape
    scrape_dict: dictionary with websites that should be scraped further (next layer)
    
    OUTPUT:
    new_dict:    a new dictionary with tuples as keys denoting the location with respect
                 to urljoin(base,extension) website and urls as values
    
    """
    if base is None and scrape_dict is None:
        raise ValueError('Neither base_link nor scrape_dict provided')
        
    if base is None or extension is None:
        raise ValueError('base and extension need to be always provided')
    
    if scrape_dict is None:
        #print('create scrape_dict')
        # initialization
        cur_url = urljoin(base,extension)
        soup = get_soup(cur_url)
        scrape_dict = get_links(soup,link_string=tuple([condition_seed]))

    # create new dict from input dict    
    new_dict = {}
    for s in list(scrape_dict.keys()):
        # get the url
        cur_url = urljoin(base,scrape_dict[s])
        soup = get_soup(cur_url)
        
        # build the link_string tuple
        critical_string = [condition_seed]
        add_string = list(s)
        critical_string = tuple(critical_string + add_string)
        cur_dict = get_links(soup,link_string=critical_string,final_layer=final_layer)
        
        new_dict.update(cur_dict)
            
    return(new_dict)

In [5]:
# when we call the function the first time (using base)
# then th first soup is scraped and the resulting dictionary immediately scraped one layer
# deeper
new_dict = scrape(base=base, scrape_dict=None,extension=extension,condition_seed='catalogue')

In [6]:
print(new_dict)

{('Ammann', 'Roller Parts'): 'index.cfm/page/catalogue/Ammann/Roller Parts', ('Atlas', 'Excavator Parts'): 'index.cfm/page/catalogue/Atlas/Excavator Parts', ('Atlas-Copco', 'Other Parts'): 'index.cfm/page/catalogue/Atlas-Copco/Other Parts', ('Bell', 'Backhoe Parts'): 'index.cfm/page/catalogue/Bell/Backhoe Parts', ('Bell', 'Grader Parts'): 'index.cfm/page/catalogue/Bell/Grader Parts', ('Bell', 'Loader Parts'): 'index.cfm/page/catalogue/Bell/Loader Parts', ('Bell', 'Off Road Truck Parts'): 'index.cfm/page/catalogue/Bell/Off Road Truck Parts', ('Bomag', 'Roller Parts'): 'index.cfm/page/catalogue/Bomag/Roller Parts', ('Doosan', 'Excavator Parts'): 'index.cfm/page/catalogue/Doosan/Excavator Parts', ('Doosan', 'Loader Parts'): 'index.cfm/page/catalogue/Doosan/Loader Parts', ('FAI', 'Excavator Parts'): 'index.cfm/page/catalogue/FAI/Excavator Parts', ('Hitachi', 'Backhoe Parts'): 'index.cfm/page/catalogue/Hitachi/Backhoe Parts', ('Hitachi', 'Crane Parts'): 'index.cfm/page/catalogue/Hitachi/Cra

In [7]:
# doing it another time...
new_dict = scrape(base=base, scrape_dict=new_dict,extension=extension,condition_seed='catalogue')

In [8]:
# another layer makes it run very slowly and the dictionary to be held in working memory
# is liable to be getting too large
# so we want to split the problem by first layer and then write to a database
all_keys = new_dict.keys()
first_layer = [i[0] for i in all_keys]
first_layer_set = list(set(first_layer))
first_layer_set.sort()
print(first_layer_set)

['Ammann', 'Atlas', 'Atlas-Copco', 'Bell', 'Bomag', 'Doosan', 'FAI', 'Hitachi', 'Hyundai', 'JCB', 'Kawasaki', 'Komatsu', 'Moxy', 'Volvo']


In [9]:
# function to split off dictionary
# according to first layer name
def get_part_dict(new_dict,first_layer_name):
    cur_dict = {}
    key_list = list(all_keys)
    for k in key_list:
        # checks the first element of the key tuple
        if k[0] == first_layer_name:
            cur_dict.update({k: new_dict[k]})
    return(cur_dict)

cur_dict = get_part_dict(new_dict,first_layer_set[2])
print(cur_dict)


{('Atlas-Copco', 'Other Parts', 'ROC F9-11'): 'index.cfm/page/catalogue/Atlas-Copco/Other Parts/ROC F9-11', ('Atlas-Copco', 'Other Parts', 'ROC L8 TH, SM'): 'index.cfm/page/catalogue/Atlas-Copco/Other Parts/ROC L8 TH, SM'}


In [10]:
for i,f in enumerate(first_layer_set):
    if i > 3:
        break
    print('Working on: ' + f)
    # split off the current smaller dictionary
    cur_dict = get_part_dict(new_dict,f)
    
    # scrape
    # this is the final layer, which changes the string affix to search for
    cur_dict = scrape(base=base, scrape_dict=cur_dict,extension=extension,condition_seed='catalogue',final_layer=True)
    
    # put the result into a data.frame
    data = []
    cur_keys = list(cur_dict.keys())
    for k in cur_keys:
        k_list = list(k)
        # split the last string variable
        # only first split will be taken into account (ot ignore ' - ' in later names)
        k_list = k_list[:-1] + k_list[-1].split(' - ',maxsplit=1)
        cur_line = k_list + [cur_dict[k]]
        data.append(cur_line)
    
    cur_df = pd.DataFrame(data)
    cur_df.columns = ['manufacturer','category','model','part','part_category','link_extension']
    
    # put into data base
    connection = sqlite3.connect("catalogue.db")
    cur_df.to_sql(con=connection,name=f,if_exists = 'replace')
    
    # clear variables
    cur_df = []
    cur_dict = {}

Working on: Ammann
Working on: Atlas
Working on: Atlas-Copco
Working on: Bell


In [11]:
# the scraping takes too long
# Hitachi has just too many subpages
# I'd have to look into time-saving / memory saving strategies
# ideas:
#  faster requests; more efficient html parser
#  somehow reduce the number of website calls
#  go directly from the final page and use the '?part=' number
#  this may be well over a million, where to start, where to end?

In [12]:
# get one of the tables
connection = sqlite3.connect("catalogue.db")

mydf = pd.read_sql(con=connection,sql='SELECT * FROM AMMANN')
print(mydf.head(10))

mydf = pd.read_sql(con=connection,sql='SELECT * FROM BELL')
print(mydf.head(10))

   index manufacturer      category   model      part part_category  \
0      0       Ammann  Roller Parts  ASC100  ND011710    LEFT COVER   
1      1       Ammann  Roller Parts  ASC100  ND011758      VIBRATOR   
2      2       Ammann  Roller Parts  ASC100  ND011785   RIGHT COVER   
3      3       Ammann  Roller Parts  ASC100  ND017673     ECCENTRIC   
4      4       Ammann  Roller Parts  ASC100  ND017675     ECCENTRIC   
5      5       Ammann  Roller Parts  ASC100  ND018184           HUB   
6      6       Ammann  Roller Parts  ASC100  ND018193       BRACKET   
7      7       Ammann  Roller Parts  ASC100  ND018214    LEFT SHAFT   
8      8       Ammann  Roller Parts  ASC100  ND018216   RIGHT SHAFT   
9      9       Ammann  Roller Parts  ASC100  ND018218         SHAFT   

                        link_extension  
0  index.cfm/page/catalogue?part=18396  
1  index.cfm/page/catalogue?part=20631  
2  index.cfm/page/catalogue?part=19582  
3  index.cfm/page/catalogue?part=17535  
4  index.cfm/

In [13]:
connection.close()